In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack, hstack
import json
import shutil

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [4]:
full_targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")
zcat = Table.read("strategy_A/zcat/dark2_dark3_zcat.fits")
full_mtl = Table.read("strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits")

ii = np.in1d(full_mtl['TARGETID'], zcat['TARGETID'])
in_mtl = full_mtl[ii]

ii = np.in1d(full_targets['TARGETID'], zcat['TARGETID'])
in_targets = full_targets[ii]

print(len(zcat), len(in_mtl), len(in_targets))
in_mtl.sort(keys='TARGETID')
zcat.sort(keys='TARGETID')
in_targets.sort(keys='TARGETID')


991020 991020 991020


In [12]:
assert np.all(in_mtl['TARGETID']==zcat['TARGETID'])
assert np.all(in_mtl['TARGETID']==in_targets['TARGETID'])

In [13]:
zcat_large = zcat[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_mtl_large = in_mtl[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_targets_large = in_targets[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
print(len(zcat_large), 100*len(zcat_large)/len(zcat))

1835 0.18516276159916045


In [14]:
names = {}
name_id = {}
for i in range(len(in_mtl_large)):
    a = desi_mask.names(in_mtl_large['DESI_TARGET'][i])
    n = ''
    for aa in a:
        n = n +' '+ aa
    if n not in names.keys():
        names[n] = 1
    else:
        names[n] += 1
    name_id[in_mtl_large['TARGETID'][i]] = n


In [17]:
names

{' ELG QSO ELG_SOUTH QSO_SOUTH': 1333,
 ' QSO QSO_SOUTH BGS_ANY': 55,
 ' QSO QSO_SOUTH': 343,
 ' QSO QSO_SOUTH BGS_ANY MWS_ANY': 78,
 ' ELG QSO ELG_SOUTH QSO_SOUTH BGS_ANY': 14,
 ' QSO QSO_SOUTH MWS_ANY': 9,
 ' QSO QSO_SOUTH STD_FAINT BGS_ANY MWS_ANY': 1,
 ' LRG QSO LRG_1PASS LRG_SOUTH QSO_SOUTH LRG_1PASS_SOUTH': 1,
 ' QSO QSO_SOUTH STD_WD BGS_ANY MWS_ANY': 1}

In [18]:
zcat_large[:10]

TARGETID,BRICKNAME,SPECTYPE,Z,ZERR,ZWARN,NUMOBS
int64,str8,str10,float32,float32,int32,int32
35185790904438384,,QSO,2.0,0.0,0,5
35185796730323060,,QSO,3.0,0.0,0,5
35185796742909236,,QSO,3.0,0.0,0,5
35185796776460872,,QSO,3.0,0.0,0,5
35185796822598387,,QSO,2.0,0.0,0,5
35185796839376327,,QSO,3.0,0.0,0,5
35185796868739057,,QSO,3.0,0.0,0,5
35185796877127721,,QSO,3.0,0.0,0,5
35185796889710154,,QSO,2.0,0.0,0,5


In [24]:
in_mtl_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
176.201380939,1.3408373188,35185790904438384,393222,0,0,3400,0.127062507512,4,26974,2,3000,3
163.443150366,1.37971461496,35185796730323060,1152921504607109124,65537,0,3400,0.0663812426865,4,27057,1,3500,7
164.228191102,1.58616573809,35185796742909236,393222,0,0,3400,0.00100722069003,4,27060,1,3500,3
166.170830527,1.40723682634,35185796776460872,393222,0,0,3400,0.556551896347,4,27039,1,3500,3
168.819575222,1.41857230733,35185796822598387,393222,0,0,3400,0.201642936455,4,27075,1,3000,3
169.888781448,1.45168720786,35185796839376327,262148,0,0,3400,0.98619510698,4,27076,2,3500,1
171.652835438,1.58829568805,35185796868739057,393222,0,0,3400,0.834028952502,4,26991,1,3500,3
172.075751519,1.62105088243,35185796877127721,393222,0,0,3400,0.53764480336,4,26989,1,3500,3
172.76469936,1.59185633603,35185796889710154,393222,0,0,3400,0.632979629017,4,27000,1,3000,3


In [25]:
in_targets_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
176.201380939,1.3408373188,35185790904438384,393222,0,0,3400,0.127062507512,4,26974,4,3400,3
163.443150366,1.37971461496,35185796730323060,1152921504607109124,65537,0,3400,0.0663812426865,4,27057,4,3400,7
164.228191102,1.58616573809,35185796742909236,393222,0,0,3400,0.00100722069003,4,27060,4,3400,3
166.170830527,1.40723682634,35185796776460872,393222,0,0,3400,0.556551896347,4,27039,4,3400,3
168.819575222,1.41857230733,35185796822598387,393222,0,0,3400,0.201642936455,4,27075,4,3400,3
169.888781448,1.45168720786,35185796839376327,262148,0,0,3400,0.98619510698,4,27076,4,3400,1
171.652835438,1.58829568805,35185796868739057,393222,0,0,3400,0.834028952502,4,26991,4,3400,3
172.075751519,1.62105088243,35185796877127721,393222,0,0,3400,0.53764480336,4,26989,4,3400,3
172.76469936,1.59185633603,35185796889710154,393222,0,0,3400,0.632979629017,4,27000,4,3400,3


In [27]:
name_id

{35185790904438384: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796730323060: ' QSO QSO_SOUTH BGS_ANY',
 35185796742909236: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796776460872: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796822598387: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796839376327: ' QSO QSO_SOUTH',
 35185796868739057: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796877127721: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796889710154: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796906486832: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796948428737: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796969400797: ' QSO QSO_SOUTH BGS_ANY MWS_ANY',
 35185796973595923: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796977789014: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802740760729: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802765930334: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802778512937: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802795287835: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802862397901: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185802862398194: ' ELG QSO 

In [28]:
def consolidate_fassigned(fba_path):
    fba_files = glob.glob(os.path.join(fba_path, 'tile-*fits'))

    # getting all the targetids of the assigned fibers
    print('reading individual fiberassign files')
    favail = list()
    for i_tile, tile_file in enumerate(fba_files):
        if i_tile%50 ==0:
            print(i_tile)
        id_favail, header = fits.getdata(tile_file, 'FIBERASSIGN', header=True)
        favail.extend(id_favail['TARGETID'])
    return np.array(favail)

In [29]:
mtl_names = {0:"strategy_A/targets/gray_subset_dr8_mtl_dark_gray_northern_cap.fits",
             1:"strategy_A/targets/dark0_subset_dr8_mtl_dark_gray_northern_cap.fits",
             2:"strategy_A/targets/dark1_subset_dr8_mtl_dark_gray_northern_cap.fits",
             3:"strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits"}
zcat_names = {0:"strategy_A/zcat/gray_zcat.fits",
             1:"strategy_A/zcat/dark0_zcat.fits",
             2:"strategy_A/zcat/dark1_zcat.fits",
             3:"strategy_A/zcat/dark2_dark3_zcat.fits"}
tile_names = {0:'strategy_A/fiberassign_gray/', 
             1:'strategy_A/fiberassign_dark0/',
             2:'strategy_A/fiberassign_dark1/',
             3:'strategy_A/fiberassign_dark2_dark3/'}

mtl_data = {}
zcat_data = {}
tile_data = {}
for k in mtl_names.keys():
    mtl_data[k] = Table.read(mtl_names[k])
    zcat_data[k] = Table.read(zcat_names[k])
    tile_data[k] = consolidate_fassigned(tile_names[k])

reading individual fiberassign files
0
reading individual fiberassign files
0
reading individual fiberassign files
0
50
reading individual fiberassign files
0
50


In [31]:
def print_stats(id_to_track):
    print("ID_TO_TRACK", id_to_track)
    for k in range(4):
        ii = mtl_data[k]['TARGETID']==id_to_track
        jj = zcat_data[k]['TARGETID']==id_to_track
        kk = np.count_nonzero(tile_data[k]==id_to_track)
        fa = Table([[kk]], names=(['NFIBER']))
        if(np.count_nonzero(jj)):
            a = hstack((mtl_data[k][['PRIORITY', 'NUMOBS_MORE']][ii], fa, zcat_data[k][['NUMOBS']][jj]))
        else:
            a = hstack((mtl_data[k][['PRIORITY', 'NUMOBS_MORE']][ii], fa))      
        print(a)
    print()

In [33]:
ids_to_track = [35185796730325505, 35186182941835863, 35186188885166096, 35186188750949832, 35186194501340726, 35186107519861231]
for id_to_track in ids_to_track:
    print_stats(id_to_track)

ID_TO_TRACK 35185796730325505
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3000           3      0      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3000           3      0      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3000           3      0      1

ID_TO_TRACK 35186182941835863
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1


PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
       2           0      0      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
       2           0      0      1

ID_TO_TRACK 35186188885166096
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      2      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      1      3
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           1      1      5

ID_TO_TRACK 35186188750949832
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3000           3      0      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3000           3      1      2
PRIORITY NUMOB